In [ ]:
import requests
import time
import pandas as pd
from scipy import stats

def UserList():
    global user_list
    for i in range(0,10): #just do the first 300 for demo. my final project will scrape about 1,000,000? Can I do this?
        time.sleep(0.2)
        user_id=i*30 #because each search result gives you 30.
        r = requests.get('https://api.github.com/users?since=%d' %user_id, auth = ('username', 'password'))
        if r.status_code == 200:
            response_data = r.json()
            for num in range(0,30):
                user_list.append(response_data[num]['login']) #i only need the login name.

def Contrib():
    global df
    for name in user_list:
        time.sleep(0.1)
        r = requests.get('https://github-contributions-api.now.sh/v1/'+str(name), auth=('username', 'password'))
        contrib_data = r.json()
        df_temp = pd.DataFrame(contrib_data['years'])
        df_temp['login_name']= name
        df = df.append(df_temp, sort=False, ignore_index=True)

user_id=0
user_list=[]
df = pd.DataFrame(columns=['range','total','year','login_name'])
UserList()
Contrib()

# now to do the t-test.
grp_2017 = df[df['year'] == '2017']
grp_2016 = df[df['year'] == '2016']
grp_2010 = df[df['year'] == '2010']
pval = stats.ttest_ind(grp_2017['total'],grp_2010['total']).pvalue

if pval < 0.05:
    print("The Github commits in 2017 are significantly different from the commits in 2010. P-value: {:.4f}".format(pval))
else:
    print("The Github commits in 2017 are not significantly different from the commits in 2010.  P-value: {:.4f}".format(pval))
